# Import Packages

In [ ]:
!pip install torchsummary

In [ ]:
import pydicom
import pandas as pd
import numpy as np
import random
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

from torch.utils.data import Dataset, Subset, DataLoader
from torchvision.transforms import v2
import torchvision.models as models # VGG16, ResNet50
from torchsummary import summary # summary for model印出模型的資料
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from collections import OrderedDict

# Configuration

In [ ]:
class config:
    
    root = '/kaggle/input/d/chieh7/hw2-data/hwk02_data'
    valid_prob = 0.2
    batch_size_VGG = 14
    batch_size_Res = 16
    lr_VGG = 1e-4
    lr_Res = 1e-4
    epochs_VGG = 300
    epochs_Res = 300
    weight_decay_VGG = 1e-5
    weight_decay_Res = 1e-5
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    seed = 42
    
print(config.device)

In [ ]:
dicom_path = '/DICOM/A1406/00010019'
dicom_file = pydicom.dcmread(config.root+dicom_path)
print(dicom_file)#包括DICOM文件中的所有標籤（例如患者信息、影像信息等）。

In [ ]:
print(dicom_file.WindowCenter, dicom_file.WindowWidth)
images = dicom_file.pixel_array
print(images.shape)
plt.figure(figsize=(16, 20))

for i in range(images.shape[0]):
    plt.subplot(5,4,i+1)
    plt.imshow(images[i]) #gray
    plt.title(f'slice {i+1}')
    plt.axis('off')
    plt.subplots_adjust(wspace=None, hspace=None)


In [ ]:
train_df = pd.read_csv(config.root+'/train.csv')
train_df
#index
test_df = pd.read_csv(config.root+'/test.csv')
test_df


# Input Data

In [ ]:
train_data = pd.read_csv(config.root+'/train.csv')
test_data = pd.read_csv(config.root+'/test.csv')
print(f'Number of training samples: {train_data.shape[0]}')
print(f'Number of testing samples: {test_data.shape[0]}')

In [ ]:
train_data['Stage'].value_counts().sort_index()

# Defined functions

In [ ]:
def seed_everything(seed):
    # Set Python random seed
    random.seed(seed)
    
    # Set NumPy random seed
    np.random.seed(seed)
    
    # Set PyTorch random seed for CPU and GPU
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    # Set PyTorch deterministic operations for cudnn backend
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def evaluator(preds, gts):
    preds = preds.cpu().numpy() if isinstance(preds, torch.Tensor) else preds
    gts = gts.cpu().numpy() if isinstance(gts, torch.Tensor) else gts
    acc = accuracy_score(preds, gts)
    f1 = f1_score(preds, gts, average="macro")

    return acc, f1

def train_one_epoch(model, train_loader, optimizer, scheduler, criterion, device):
    model.train()
    train_loss = .0
    predictions, ground_truths = [], []
    for images, ages, genders, labels in train_loader:
        images = images.to(device=device, dtype=torch.float)
        ages = ages.to(device=device, dtype=torch.float)
        genders = genders.to(device=device, dtype=torch.float)
        labels = labels.to(device=device, dtype=torch.long)

        optimizer.zero_grad()
        logits = model(images, ages, genders)
        labels = torch.sub(labels, 1) # labels 1, 2, 3 -> 0, 1, 2
        loss = criterion(logits, labels)
        loss.backward()

        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        preds = torch.argmax(logits, dim=1)

        predictions.append(preds)
        ground_truths.append(labels)

    train_loss /= len(train_loader)

    predictions = torch.cat(predictions)
    ground_truths = torch.cat(ground_truths)
    train_acc, train_f1 = evaluator(predictions, ground_truths)

    return train_loss, 100*train_acc, 100*train_f1

def validation(model, valid_loader, criterion, device):
    model.eval()
    valid_loss = .0
    predictions, ground_truths = [], []
    with torch.no_grad():
        for images, ages, genders, labels in valid_loader:
            images = images.to(device=device, dtype=torch.float)
            ages = ages.to(device=device, dtype=torch.float)
            genders = genders.to(device=device, dtype=torch.float)
            labels = labels.to(device=device, dtype=torch.long)

            logits = model(images, ages, genders)
            labels = torch.sub(labels, 1) # labels 1, 2, 3 -> 0, 1, 2
            loss = criterion(logits, labels)

            valid_loss += loss.item()
            preds = torch.argmax(logits, dim=1)

            predictions.append(preds)
            ground_truths.append(labels)

        valid_loss /= len(valid_loader)

        predictions = torch.cat(predictions)
        ground_truths = torch.cat(ground_truths)
        valid_acc, valid_f1 = evaluator(predictions, ground_truths)
    return valid_loss, 100*valid_acc, 100*valid_f1

def test(model,test_loader, device):
    model.eval()
    predictions_pro = []
    predictions_stage = []
    with torch.no_grad():
        for images, ages, genders,labels in test_loader:
            images = images.to(device=device, dtype=torch.float)
            ages = ages.to(device=device, dtype=torch.float)
            genders = genders.to(device=device, dtype=torch.float)
            labels = labels.to(device=device, dtype=torch.long)
            logits = model(images, ages, genders)
            pred_pro = nn.functional.softmax(logits, dim=1)
            pred_stage = torch.argmax(pred_pro, dim=1)
            predictions_pro.append(pred_pro.cpu().numpy())
            predictions_stage.append((pred_stage.cpu().numpy()+1))
             
    predictions_pro = np.concatenate(predictions_pro, axis=0)
    predictions_stage = np.concatenate(predictions_stage, axis=0)
   
    
    return  predictions_pro, predictions_stage

# Defined Dataset

In [ ]:
class ParkinsonsDataset(Dataset):
    def __init__(self, df, transforms = None): # 將所有資料提出
        self.ages = np.array(df['Age']) # 年齡
        self.genders = np.array(df['Gender']) # 性別
        self.labels = np.array(df['Stage']) # 標籤
        self.indexs = np.array(df['index']) # 起始張數
        self.paths = np.array(df['FilePath']) # 影像路徑
        self.images = []

        # 影像前處理
        for index, path in zip(self.indexs, self.paths):
            image = pydicom.dcmread(config.root+path).pixel_array
            image = torch.tensor(image.astype(np.float32))
            image = image[index-1:index+2, :, :] # 取指定張數和前後共三個, image size = (3, 128, 128)
            if transforms:
                image = transforms(image)
            self.images.append(image)


    def __len__(self):
        return len(self.labels) # label數量

    def __getitem__(self, idx): # 找出指定的資料
        age = torch.tensor(self.ages[idx], dtype=torch.float32)
        gender = torch.tensor(self.genders[idx], dtype=torch.float32)
        label = self.labels[idx]
        image = self.images[idx]

        return image, age, gender, label

# Transformation
class My_normalize(object): # 自定義的類別
    # def __init__(self): # 宣告此類別中會使用到的變數

    def __call__(self, image): # 定義有image之後要進行的資料處理
        new_image = (image - image.mean())/(image.max() - image.min())

        return new_image

My_transforms = v2.Compose([
    # 在v2中內建的func
    v2.CenterCrop(size = (50, 50)),
    # 自定義的func
    My_normalize()
])

In [ ]:
dataset = ParkinsonsDataset(train_df, transforms = My_transforms)
test_dataset = ParkinsonsDataset(test_df, transforms = My_transforms)
print(f"Number of training samples: {len(dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

train_labels_distribution = Counter(dataset.labels)#使用 Counter 函式計算資料集中各標籤的數量
print(train_labels_distribution)
x = [key.astype(str) for key in train_labels_distribution.keys()]
y = [value for value in train_labels_distribution.values()]
plt.bar(x, y, color = 'lightcoral')
plt.ylabel("Number of samples")
plt.title("Distribution of labels in training dataset")
plt.xticks()#設定 x 軸的刻度標籤
plt.show()

In [ ]:
subdf = train_df[(train_df['Gender'] == 1) & (train_df['Stage'] == 1)]
subdf.shape[0]
male = [train_df[(train_df['Gender'] == 1) & (train_df['Stage'] == i)].shape[0] for i in range(1, 4)]
female = [train_df[(train_df['Gender'] == 0) & (train_df['Stage'] == i)].shape[0] for i in range(1, 4)]
print(male, female)
x_axis = np.arange(len(x))
plt.bar(x_axis - 0.2, male, 0.4, label = 'Male', color = 'cornflowerblue')
plt.bar(x_axis + 0.2, female, 0.4, label = 'Female', color = 'lightcoral')
plt.xticks(x_axis, x)
plt.xlabel("Stages")
plt.ylabel("Number of samples")
plt.title("Number of samples in each group")
plt.legend()
plt.show()

In [ ]:
sub_df = [train_df[train_df['Stage'] == i]['Age'] for i in range(1, 4)]
plt.boxplot(sub_df)
plt.xlabel("Stages")
plt.ylabel("Number of samples")
plt.title("Number of samples in each group")
plt.show()


In [ ]:
all_data = [np.random.normal(0, std, size=100) for std in range(1, 4)]
len(all_data)
n = len(dataset)
valid_size = int(n * 0.3) # 取30%作為validation
train_ids , vaild_ids = train_test_split(
    np.linspace(0, n - 1, n).astype("int"),
    test_size = valid_size,
    random_state=42,
)
train_dataset = Subset(dataset, train_ids)
val_dataset = Subset(dataset, vaild_ids)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
for images, ages, genders, labels in train_loader:
        images = images.to(device=config.device, dtype=torch.float)
        ages = ages.to(device=config.device, dtype=torch.float)
        genders = genders.to(device=config.device, dtype=torch.float)
        labels = labels.to(device=config.device, dtype=torch.long)
        print(images.shape)
        print(images)
        #print(labels)
        #print("torch")
        #labels = torch.sub(labels,0) # labels 1, 2, 3 -> 0, 1, 2
        #print(labels)

In [ ]:
images, ages, genders, labels = next(iter(train_loader))

plt.figure(figsize=(12, 64)) # (寬, 長)

for i, (image ,label) in enumerate(zip(images, labels)):
    for j in range(3):
        plt.subplot(16, 3,i*3+j+1)
        plt.title(label)
        plt.imshow(image[j])
        plt.axis('off')
        plt.subplots_adjust(wspace=None, hspace=None)

print(f"Image shape: {images.shape}, label shape {label.shape}")

# Redefined VGG16

In [ ]:
class VGGplus(nn.Module):
    def __init__(self, num_classes, input_size = (3, 50, 50), out_prob = True, features_grad = False): # 默認輸出為分到每一類的機率
        super().__init__()
        self.max_grad_norm = 0.1  
        # 決定是否要將輸出轉換為機率
        self.out_prob = out_prob
        
        # 取出vgg16中的特徵層
        vgg16 = models.vgg16(weights='IMAGENET1K_V1', progress = True)
        vgg16.features[30] = nn.Identity()
        vgg16.avgpool = nn.AdaptiveAvgPool2d(1)
        vgg16.classifier = nn.Identity()
        
        # 固定/不固定特徵層的參數值
        for param in vgg16.features.parameters():
            param.requires_grad = features_grad
        self.backend = vgg16
         
        # 增加分類層
        self.classifier = nn.Sequential(
          nn.Linear(514, num_classes) # 512: vgg16特徵層結果, 2: age & gender
        )
        self.softmax = nn.Softmax(dim=1) # 每一個row的總和都是1
       
        for param in self.classifier.parameters():
            param.requires_grad = True  

    def forward(self, x, age, gender):
        output = self.backend(x)
        outputs = torch.cat([output, age.view(-1, 1), gender.view(-1, 1)], dim = 1) # output size = (batch_size, 512), age size = (batch_size), age.view(-1, 1) size = (batch_size, 1), dim = 1; columns concat
        outputs = self.classifier(outputs) # outputs size = (batch_size, 3)
        if self.out_prob:
            outputs = self.softmax(outputs)
            
        torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=self.max_grad_norm)
        return outputs

In [ ]:
model = VGGplus(num_classes = 3)
print(model)
del model

### 1. 導入 vgg16 的架構和 ImageNet 訓練出來的權重 (**transfer learning**)

In [ ]:
vgg16 = models.vgg16(weights='IMAGENET1K_V1', progress = True)
print(vgg16)

以下簡單介紹vgg16中使用到的函式: (以下函式的輸入都需要四個維度: (N = number of data, C = number of channels, H = Height, W = Width))

> [Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

> * 3: in_channels
> * 64: out_channels 
> * stride = (1, 1): kernel的移動由左至右, 由上至下, 步長皆為1
> * padding = (1, 1): 在輸入的周圍填充一層0

> Note:
> * 雖然是2d的convolution, 但實際的輸入是和kernel都是3維陣列, 而2d指的是kernel移動的方向。
> * 因為1個(3, 3, 3)的kernel和1個bias遍歷一次輸入就會產生一層輸出, 這邊的輸出有64層, 就需要64個(3, 3, 3)的kernal和對應的64個bias。
> * 這樣的stride和padding的設定可以讓輸出和輸入的長寬不變。

![](https://discuss.pytorch.org/uploads/default/original/3X/6/0/60994f2ed1f8c34ee6e83741e2e87fca0a1b4655.jpeg)

In [ ]:
m = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
print(m)
input = torch.randn(1, 3, 50, 50)
print(input.shape)
output = m(input)
print(output.shape)
for param in m.parameters():
    print(param.shape)
    
del m, input, output

> [MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d)(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

> * ceil_mode: True = 將不足的邊用NAN補齊, False = 將不足的邊直接刪除

In [ ]:
input = torch.tensor([[1, 1, 1], [2, 2, 2], [3, 3, 3]]).reshape(1, 1, 3, 3).float()
print(input)

print("ceil_mode = False")
m1 = nn.MaxPool2d(2, stride=2, ceil_mode = False)
output1 = m1(input)
print(output1)

print("ceil_mode = True")
m2 = nn.MaxPool2d(2, stride=2, ceil_mode = True)
output2 = m2(input)
print(output2)

del m1, m2, input, output1, output2

> [AdaptiveAvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html)(output_size=(7, 7)) 

> 按照期望的輸出大小對輸入做平均

In [ ]:
input = torch.tensor([[1, 1, 1], [2, 2, 2], [3, 3, 3]]).reshape(1, 1, 3, 3).float()
print(input)

m7 = nn.AdaptiveAvgPool2d(output_size=(7, 7))
output = m7(input)
print(output)

m2 = nn.AdaptiveAvgPool2d(output_size=(2, 2))
output = m2(input)
print(output)

del input, m7, m2, output

雖然AdaptiveAvgPool2d()的輸出大小可以大於輸入大小，但這種作法可能導致訊息的丟失。

### 2. 觀察 vgg16 的 Output Shape 去設計 feature layers 和 classifier layers 之間的連接層

In [ ]:
# 因summary()的input存在GPU中, 要將model也移至GPU上才能運行

vgg16.to(config.device)
summary(vgg16, input_size = (3, 50, 50)) 

在ReLU-30得到的輸出大小已經是(512, 3, 3), 如果使用模型中的MaxPool2d-31:MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, **ceil_mode=False**), 會有些資訊被忽略, 所以接下來會更改31層和32層。

In [ ]:
vgg16.features[30] = nn.Identity()
vgg16.avgpool = nn.AdaptiveAvgPool2d(1)
vgg16.classifier = nn.Identity()
summary(vgg16, (3, 50, 50))

Note: 
* 在avgpool和classifier之間, vgg16模型會把(-1, 512, 1, 1)reshape成(-1, 512)。
* 每一個(3, 50, 50)的影像經過更改後的vgg16模型就會產生512個數值。

### 3. 增加影像之外的變數(ex. 性別, 年齡, ...)去建立 classifier layers

這裡想建立一個特徵層可以考慮剛剛影像經過特徵層所得到的512個數值和性別, 年齡這兩個數值, 所以分類層的一開始會有512+2 = 514個神經元。

In [ ]:
del vgg16

In [ ]:
class ResNetplus(nn.Module):
    def __init__(self, num_classes, input_size = (3, 50, 50), out_prob = True, features_grad = False): # 默認輸出為分到每一類的機率
        super().__init__()
        self.max_grad_norm = 0.1  # 設定梯度裁剪的閾值
       
        
        # 決定是否要將輸出轉換為機率
        self.out_prob = out_prob
        
        # 取出vgg16中的特徵層
        resnet = models.resnet50(weights='IMAGENET1K_V1', progress = True)

        resnet = nn.Sequential(*list(resnet.children())[:-2])
        # Add your custom layers
        custom_layers = [
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(2048, 1024),
            nn.ReLU()]
        custom_layers = nn.Sequential(*custom_layers)
        # Create a new Sequential model by combining resnet and custom_layers
        resnet = nn.Sequential(resnet, custom_layers)



        # 固定/不固定特徵層的參數值 
        for param in resnet.parameters():
            param.requires_grad = features_grad
        self.backend = resnet 

        # 增加分類層
        self.classifier = nn.Sequential(
          nn.Linear(1026, num_classes) 
        )
        self.softmax = nn.Softmax(dim=1) # 每一個row的總和都是1
        
       
       

    def forward(self, x, age, gender):
        output = self.backend(x)
        outputs = torch.cat([output, age.view(-1, 1), gender.view(-1, 1)], dim = 1) # output size = (batch_size, 512), age size = (batch_size), age.view(-1, 1) size = (batch_size, 1), dim = 1; columns concat
        outputs = self.classifier(outputs) # outputs size = (batch_size, 3)
        if self.out_prob:
            outputs = self.softmax(outputs)
        # 执行梯度裁剪
       
        torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=self.max_grad_norm)

        return outputs

In [ ]:
""""
resnet = models.resnet50(weights='IMAGENET1K_V1', progress = True)
model_state = resnet.state_dict()
print("model_state type:", type(model_state))
for param_tensor in model_state:
    print("name:", param_tensor)
    print("value:", model_state[param_tensor])
    """

In [ ]:
resnet = models.resnet50(weights='IMAGENET1K_V1', progress = True)
resnet = nn.Sequential(*list(resnet.children())[:-2])

# Add your custom layers
custom_layers = [
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.3)
]
custom_layers = nn.Sequential(*custom_layers)

# Create a new Sequential model by combining resnet and custom_layers
resnet = nn.Sequential(resnet, custom_layers)



resnet.to(config.device)
summary(resnet, input_size = (3, 50, 50))

In [ ]:
# train & test dataframe
    train_df = pd.read_csv(config.root+'/train.csv')
    test_df = pd.read_csv(config.root+'/test.csv')

   
    dataset = ParkinsonsDataset(train_df, transforms = My_transforms)
  
    test_dataset = ParkinsonsDataset(test_df, transforms = My_transforms)
    
    # split training & validation dataset 
    n = len(dataset)
    valid_size = int(n * config.valid_prob)
    train_ids , valid_ids = train_test_split(
     np.linspace(0, n - 1, n).astype("int"),
     test_size = valid_size,
     random_state = config.seed,
    )
    print(f'Number of samples in train_dataset: {Counter(dataset.labels[train_ids])}')
    print(f'Number of samples in val_dataset: {Counter(dataset.labels[valid_ids])}')
    
    # DataLoader
    train_dataset = Subset(dataset, train_ids)
    valid_dataset = Subset(dataset, valid_ids)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size_VGG, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=config.batch_size_VGG, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size_VGG, shuffle=False)

In [ ]:
def VGG_modeling():

    seed_everything(config.seed)

    # train & test dataframe
    train_df = pd.read_csv(config.root+'/train.csv')
    test_df = pd.read_csv(config.root+'/test.csv')

    # Dataset
    print("VGG16")
    print("Initializing dataset...")
    dataset = ParkinsonsDataset(train_df, transforms = My_transforms)
    print("Initializing test_dataset...")
    test_dataset = ParkinsonsDataset(test_df, transforms = My_transforms)
    
    # split training & validation dataset 
    n = len(dataset)
    valid_size = int(n * config.valid_prob)
    train_ids , valid_ids = train_test_split(
     np.linspace(0, n - 1, n).astype("int"),
     test_size = valid_size,
     random_state = config.seed,
    )
    print(f'Number of samples in train_dataset: {Counter(dataset.labels[train_ids])}')
    print(f'Number of samples in val_dataset: {Counter(dataset.labels[valid_ids])}')
    
    # DataLoader
    train_dataset = Subset(dataset, train_ids)
    valid_dataset = Subset(dataset, valid_ids)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size_VGG, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=config.batch_size_VGG, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size_VGG, shuffle=False)

    # settings
    print("VGG16 training")
    print("Initializing model...")
    num_classes = len(Counter(dataset.labels[train_ids]))
    model = VGGplus(num_classes = num_classes, features_grad = True)
    model.to(config.device)
    criterion = nn.CrossEntropyLoss().to(config.device)
    optimizer = torch.optim.Adam(model.parameters(), lr = config.lr_VGG, weight_decay = config.weight_decay_VGG)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer = optimizer,
        epochs = config.epochs_VGG,
        steps_per_epoch = train_loader.__len__(),
        max_lr = config.lr_VGG,
        anneal_strategy = 'cos'
    )

    # recordings
    best_val_loss = float("inf")
    history = {
      "train": {
          "loss": [],
          "acc": [],
          "f1": []
      },
      "valid": {
          "loss": [],
          "acc": [],
          "f1": []
      },
    }
    
    for epoch in range(config.epochs_VGG):
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer, scheduler, criterion, config.device)
        valid_loss, valid_acc, valid_f1 = validation(model, valid_loader, criterion, config.device)
        
        # Log the loss and validation result
        history["train"]["loss"].append(train_loss)
        history["train"]["acc"].append(train_acc)
        history["train"]["f1"].append(train_f1)
        history["valid"]["loss"].append(valid_loss)
        history["valid"]["acc"].append(valid_acc)
        history["valid"]["f1"].append(valid_f1)

        print(f'Epoch[{epoch+1}/{config.epochs_VGG}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, Train F1: {train_f1:.2f}% | Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_acc:.2f}%, Valid F1: {valid_f1:.2f}% | LR: {optimizer.state_dict()["param_groups"][0]["lr"]:.6f}')

        if valid_loss < best_val_loss:
            save_file = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
                "epoch": epoch,
                "args": config
            }
            best_val_loss = valid_loss
            torch.save(save_file, "checkpoint.pth")
            
    best_ckpt = torch.load("checkpoint.pth", map_location=config.device)
    model.load_state_dict(best_ckpt["model"])

    print("VGG plot")
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_VGG), history["train"]["loss"], label='Training Loss')
    plt.plot(range(config.epochs_VGG), history["valid"]["loss"], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss Curves')
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_VGG), history["train"]["acc"], label='Training Acc')
    plt.plot(range(config.epochs_VGG), history["valid"]["acc"], label='Validation Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Accuracy Curves')
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_VGG), history["train"]["f1"], label='Training F1')
    plt.plot(range(config.epochs_VGG), history["valid"]["f1"], label='Validation F1')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation F1 Score Curves')
    plt.show()
    
    test_prediction_pro, test_prediction_stage = test(model, test_loader, config.device)
    # 读取现有的test.csv文件
    test_df = pd.read_csv('/kaggle/input/d/chieh7/hw2-data/hwk02_data/test.csv')

    # 创建一个DataFrame包含预测结果
    predictions = pd.DataFrame(test_prediction_pro, columns=[f'Stage({i+1})' for i in range(3)])
    predictions['Stage'] = test_prediction_stage
    # 将预测结果添加到现有DataFrame中
    test_df['Stage 1'] = predictions['Stage(1)']
    test_df['Stage 2'] = predictions['Stage(2)']
    test_df['Stage 3'] = predictions['Stage(3)']
    test_df['Stage'] = predictions['Stage']
    # 另存为新的CSV文件
    test_df.to_csv('VGG16.csv', index=False)
    print("VGG16 save")



In [ ]:
test_df = pd.read_csv(config.root+'/test.csv')
test_dataset = ParkinsonsDataset(test_df, transforms = My_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size_VGG, shuffle=False)
with torch.no_grad():
    for batch in test_loader:
        images, ages, genders, labels = batch  # 解包四个值
        # 打印一些信息以调试
        print(images.shape, ages.shape, genders.shape, labels.shape)

In [ ]:
def RseNet_modeling():

    seed_everything(config.seed)

    # train & test dataframe
    train_df = pd.read_csv(config.root+'/train.csv')
    test_df = pd.read_csv(config.root+'/test.csv')

    # Dataset
    print("ResNet 50")
    print("Initializing dataset...")
    dataset = ParkinsonsDataset(train_df, transforms = My_transforms)
    print("Initializing test_dataset...")
    test_dataset = ParkinsonsDataset(test_df, transforms = My_transforms)
    
    # split training & validation dataset 
    n = len(dataset)
    valid_size = int(n * config.valid_prob)
    train_ids , valid_ids = train_test_split(
     np.linspace(0, n - 1, n).astype("int"),
     test_size = valid_size,
     random_state = config.seed,
    )
    print(f'Number of samples in train_dataset: {Counter(dataset.labels[train_ids])}')
    print(f'Number of samples in val_dataset: {Counter(dataset.labels[valid_ids])}')
    
    # DataLoader
    train_dataset = Subset(dataset, train_ids)
    valid_dataset = Subset(dataset, valid_ids)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size_Res, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=config.batch_size_Res, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size_Res, shuffle=False)

    # settings
    print("ResNet50 training")
    print("Initializing model...")
    num_classes = len(Counter(dataset.labels[train_ids]))
    model =  ResNetplus(num_classes = num_classes, features_grad = True)
    model.to(config.device)
    criterion = nn.CrossEntropyLoss().to(config.device)
    optimizer = torch.optim.Adam(model.parameters(), lr = config.lr_Res, weight_decay = config.weight_decay_Res)
    
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer = optimizer,
        epochs = config.epochs_Res,
        steps_per_epoch = train_loader.__len__(),
        max_lr = config.lr_Res,
        anneal_strategy = 'cos'
    )

    # recordings
    best_val_loss = float("inf")
    history = {
      "train": {
          "loss": [],
          "acc": [],
          "f1": []
      },
      "valid": {
          "loss": [],
          "acc": [],
          "f1": []
      },
    }

    for epoch in range(config.epochs_Res):
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer, scheduler, criterion, config.device)
        valid_loss, valid_acc, valid_f1 = validation(model, valid_loader, criterion, config.device)
        
        # Log the loss and validation result
        history["train"]["loss"].append(train_loss)
        history["train"]["acc"].append(train_acc)
        history["train"]["f1"].append(train_f1)
        history["valid"]["loss"].append(valid_loss)
        history["valid"]["acc"].append(valid_acc)
        history["valid"]["f1"].append(valid_f1)

        print(f'Epoch[{epoch+1}/{config.epochs_Res}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, Train F1: {train_f1:.2f}% | Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_acc:.2f}%, Valid F1: {valid_f1:.2f}% | LR: {optimizer.state_dict()["param_groups"][0]["lr"]:.6f}')

        if valid_loss < best_val_loss:
            save_file = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
                "epoch": epoch,
                "args": config
            }
            best_val_loss = valid_loss
            torch.save(save_file, "checkpoint2.pth")

    best_ckpt = torch.load("checkpoint2.pth", map_location=config.device)
    model.load_state_dict(best_ckpt["model"])

    print("ResNet 50 plot")
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_Res), history["train"]["loss"], label='Training Loss')
    plt.plot(range(config.epochs_Res), history["valid"]["loss"], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss Curves')
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_Res), history["train"]["acc"], label='Training Acc')
    plt.plot(range(config.epochs_Res), history["valid"]["acc"], label='Validation Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Accuracy Curves')
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_Res), history["train"]["f1"], label='Training F1')
    plt.plot(range(config.epochs_Res), history["valid"]["f1"], label='Validation F1')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation F1 Score Curves')
    plt.show()
    
    test_prediction_pro, test_prediction_stage =  test(model, test_loader, config.device)
  
    # 读取现有的test.csv文件
    test_df = pd.read_csv('/kaggle/input/d/chieh7/hw2-data/hwk02_data/test.csv')

    # 创建一个DataFrame包含预测结果
    predictions = pd.DataFrame(test_prediction_pro, columns=[f'Stage({i+1})' for i in range(3)])
    predictions['Stage'] = test_prediction_stage
    # 将预测结果添加到现有DataFrame中
    test_df['Stage 1'] = predictions['Stage(1)']
    test_df['Stage 2'] = predictions['Stage(2)']
    test_df['Stage 3'] = predictions['Stage(3)']
    test_df['Stage'] = predictions['Stage']
    # 另存为新的CSV文件
    test_df.to_csv('ResNet50.csv', index=False)
    print("ResNet save")



In [ ]:
def main():
    VGG_modeling()
    RseNet_modeling()
    
    
if __name__ == "__main__":
    main()